# 🧪 Exemplo de Aplicação com Conexão a Banco de Dados

Este notebook demonstra como criar uma aplicação simples em Python que interage com um banco de dados **PostgreSQL** utilizando bibliotecas como **Pandas**, **SQLAlchemy**, **Panel**, entre outras. A interface gráfica permite consultar, inserir, atualizar e excluir registros da tabela `pessoa`.

---

## 🛠️ Organização do Projeto e Ambiente Virtual

Para garantir isolamento e facilitar a manutenção do ambiente Python, é **fortemente recomendado** utilizar um **ambiente virtual**. Isso evita conflitos entre dependências de diferentes projetos.

### ✅ Criar ambiente virtual (Linux, macOS ou WSL)

```bash
python3 -m venv venv
source venv/bin/activate
```

### ✅ Criar ambiente virtual (Windows)

```bash
python -m venv venv
venv\Scripts\activate
```

---

## 📦 `requirements.txt` — Instalação de Dependências

Crie um arquivo chamado `requirements.txt` no diretório do projeto com o seguinte conteúdo:

```txt
pandas
sqlalchemy
psycopg2-binary
panel
python-dotenv
```

### ✅ Instalar as dependências com o pip

```bash
pip install -r requirements.txt
```

---

## 🔐 Utilizando o `.env` para Conexão com o Banco de Dados

Para proteger informações sensíveis como usuário, senha e nome do banco, recomendamos armazenar esses dados em um **arquivo `.env`**, que não deve ser incluído no repositório de código (como o GitHub).

### ✅ Exemplo de conteúdo do arquivo `.env`

```dotenv
DB_HOST=localhost
DB_NAME=fbd-conexao
DB_USER=postgres
DB_PASS=root
```

---

## 📎 `.env.example`: Informando a Estrutura Esperada

Crie também um arquivo chamado **`.env.example`**, que serve como modelo para outras pessoas saberem quais variáveis são esperadas no projeto (sem conter dados reais).

Esse arquivo **pode ser incluído no repositório**, pois não contém credenciais, apenas a estrutura necessária.

---

## 🚫 Protegendo Dados com `.gitignore`

Adicione os seguintes itens no seu arquivo `.gitignore` para evitar subir arquivos sensíveis ao repositório:

---

## 🧑‍💻 Rodando a Aplicação

Após configurar o banco de dados, instalar as dependências e ativar o ambiente virtual, você poderá executar a aplicação com:

```bash
panel serve nome_do_arquivo.py --autoreload --show
```

Ou, se estiver usando Jupyter Notebook, poderá importar as funções diretamente e utilizar a interface com `pn.panel(...)`.

---

In [ ]:
# Importa as bibliotecas
# Validar o mínimo de CPF
# Criar uma home direcionando para o crud e para o gráfico

import os
from dotenv import load_dotenv

import pandas as pd
import psycopg2 as pg
import sqlalchemy
from sqlalchemy import create_engine
import panel as pn
import matplotlib.pyplot as plt

In [ ]:
# Carrega as variáveis do arquivo .env

load_dotenv()

In [ ]:
# Lê as variáveis de ambiente

DB_HOST = os.getenv('DB_HOST')
DB_NAME = os.getenv('DB_NAME')
DB_USER = os.getenv('DB_USER')
DB_PASS = os.getenv('DB_PASS')

In [ ]:
# Cria conexão com psycopg2 usando as variáveis carregadas

con = pg.connect(host=DB_HOST, dbname=DB_NAME, user=DB_USER, password=DB_PASS)

In [ ]:
# Define a string de conexão para o SQLAlchemy, utilizando as variáveis do .env
# Cria o objeto engine do SQLAlchemy que será usado para conectar e executar comandos no banco

cnx = f'postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}/{DB_NAME}'

sqlalchemy.create_engine(cnx)

In [ ]:
# Executa a consulta SQL para buscar todos os 
# registros da tabela 'pessoa' no banco PostgreSQL 
# e carrega o resultado em um DataFrame do pandas


query = "select * from pessoa;" 
df = pd.read_sql_query(query, cnx)

df.head()

In [ ]:
# Inicializa as extensões do Panel necessárias:
# - Tabulator para tabelas interativas
# - Notificações na interface

pn.extension()
pn.extension('tabulator')
pn.extension(notifications=True)

In [ ]:
# Criação da Home sem conectar os cliques ainda
btn_go_crud = pn.widgets.Button(name="📋 Ir para CRUD", button_type="primary", width=250)
btn_go_graficos = pn.widgets.Button(name="📊 Ir para Gráficos", button_type="success", width=250)
# btn_voltar_home = pn.widgets.Button(name='⬅ Voltar para Home', button_type='default')

In [ ]:
# Container principal da aplicação
main_area = pn.Column()

def create_home_view():
    home = pn.Column(
        pn.pane.Markdown("## 🏠 Tela Inicial"),
        pn.pane.Markdown("Escolha uma opção abaixo:"),
        pn.Spacer(height=20),
        btn_go_crud,
        btn_go_graficos,
        align="center",
        sizing_mode="stretch_width",
        margin=(50,50,50,50)
    )
    return home


In [ ]:
# Funções de navegação
def go_to_home():
    main_area.clear()
    main_area.append(create_home_view())

In [ ]:
def create_btn_voltar():
    btn_voltar_home = pn.widgets.Button(name='⬅ Voltar para Home', button_type='default')
    btn_voltar_home.on_click(lambda e: main_area.clear() or main_area.append(create_home_view()))
    return btn_voltar_home

In [ ]:
# Variável auxiliar para consultas sem filtro
flag=''

# Widgets de entrada de dados
def create_form_widgets():
    return {
        "nome": pn.widgets.TextInput(
            name="Nome",
            placeholder="Digite o nome",
            sizing_mode="stretch_width"
        ),
        "cpf": pn.widgets.TextInput(
            name="CPF",
            placeholder="Digite o CPF",
            sizing_mode="stretch_width"
        ),
        "datanasc": pn.widgets.DatePicker(name="Data de Nascimento"),
        "sexo": pn.widgets.RadioBoxGroup(
            name="Sexo",
            options=["Não Informado", "M", "F"],
            inline=True
        ),
        "salario": pn.widgets.FloatInput(
            name="Salário",
            value=1000,
            start=0
        )
    }


In [ ]:
# Botões de ações CRUD
buttonConsultar = pn.widgets.Button(name='Consultar', button_type='primary')
buttonInserir = pn.widgets.Button(name='Inserir', button_type='success')
buttonExcluir = pn.widgets.Button(name='Excluir', button_type='danger')
buttonAtualizar = pn.widgets.Button(name='Atualizar', button_type='warning')

# Botão para voltar à Home (na tela CRUD)
buttonVoltar = pn.widgets.Button(name='⬅ Voltar para Home', button_type='default')


In [ ]:
def queryAll():
    """
    Consulta todos os registros da tabela 'pessoa' e retorna um Tabulator.
    """
    df = pd.read_sql_query("SELECT * FROM pessoa", cnx)
    return pn.widgets.Tabulator(df, show_index=False, sizing_mode='stretch_width', height=400)

def on_consultar(w):
    """Consulta pelo CPF ou retorna todos."""
    try:
        cpf = w["cpf"].value
        query = f"SELECT * FROM pessoa WHERE ('{cpf}'='{flag}' OR cpf='{cpf}')"
        df = pd.read_sql_query(query, cnx)
        return pn.widgets.Tabulator(df, show_index=False, sizing_mode='stretch_width', height=400)
    except Exception as e:
        return pn.pane.Alert(f'Erro na consulta: {str(e)}', alert_type='danger')

def on_inserir(w):
    """Insere um registro novo."""
    try:
        cursor = con.cursor()
        cursor.execute(
            "INSERT INTO pessoa(nome, cpf, nascimento, sexo, salario) VALUES (%s,%s,%s,%s,%s)",
            (w["nome"].value, w["cpf"].value, w["datanasc"].value, w["sexo"].value, w["salario"].value)
        )
        con.commit()
        cursor.close()
        return queryAll()
    except Exception as e:
        cursor.execute("ROLLBACK")
        cursor.close()
        return pn.pane.Alert(f'Não foi possível inserir: {str(e)}', alert_type='danger')

def on_atualizar(w):
    """Atualiza nome e salário pelo CPF."""
    try:
        cursor = con.cursor()
        cursor.execute(
            "UPDATE pessoa SET nome=%s, salario=%s WHERE cpf=%s",
            (
                w["nome"].value,
                w["salario"].value,
                w["cpf"].value
            )
        )
        con.commit()
        cursor.close()
        return queryAll()
    except Exception as e:
        cursor.execute("ROLLBACK")
        cursor.close()
        return pn.pane.Alert(f'Não foi possível atualizar: {str(e)}', alert_type='danger')

def on_excluir(w):
    """Exclui registro pelo CPF."""
    try:
        cursor = con.cursor()
        cursor.execute("DELETE FROM pessoa WHERE cpf=%s", (w["cpf"].value,))
        con.commit()
        cursor.close()
        return queryAll()
    except Exception as e:
        cursor.execute("ROLLBACK")
        cursor.close()
        return pn.pane.Alert(f'Não foi possível excluir: {str(e)}', alert_type='danger')


In [ ]:
def table_creator(cons, ins, atu, exc, w):
    """
    Recebe booleans dos botões e executa a ação correspondente.
    """
    if cons: return on_consultar(w)
    if ins: return on_inserir(w)
    if atu: return on_atualizar(w)
    if exc: return on_excluir(w)


In [ ]:
# Cria uma ligação interativa (bind) entre os botões e a função que executa a ação correspondente,
# atualizando a tabela na interface sempre que algum botão for clicado.

# Conecta os botões à tabela interativa
form_widgets = create_form_widgets()

def create_interactive_table():
    return pn.bind(
        table_creator,
        buttonConsultar,
        buttonInserir,
        buttonAtualizar,
        buttonExcluir,
        form_widgets
    )


In [ ]:
# Tela de CRUD

# Monta o layout da interface com Panel:
# - Coluna esquerda com o título, os campos de entrada e os botões de ação
# - Coluna direita com a tabela interativa que mostra os dados do banco
# O método `.servable()` permite que essa interface seja exibida ao rodar o Panel server

def create_crud_view(w):
    """
    Cria a tela do CRUD com layout bonito.
    """
    crud_layout = pn.Row(
        pn.Column(
            pn.pane.Markdown("### 📋 Funcionário CRUD"),
            w["nome"],
            w["cpf"],
            w["datanasc"],
            w["sexo"],
            w["salario"],
            pn.Row(buttonConsultar, buttonInserir),
            pn.Row(buttonAtualizar, buttonExcluir),
            pn.Spacer(height=10),
            create_btn_voltar(),
            sizing_mode='stretch_width',
            margin=(20, 20, 20, 20)
        ),
        pn.Column(create_interactive_table(), sizing_mode='stretch_both')
    )
    
    return crud_layout


In [ ]:
def go_to_crud():
    main_area.clear()
    main_area.append(create_crud_view(form_widgets))

In [ ]:
# Tela de Gráficos

def grafico_salario_por_sexo():
    df = pd.read_sql_query("SELECT sexo, AVG(salario) AS media_salario FROM pessoa GROUP BY sexo", cnx)
    fig, ax = plt.subplots()
    ax.bar(df["sexo"], df["media_salario"], color=['#1f77b4', '#ff7f0e', '#2ca02c'])
    ax.set_title("Salário Médio por Sexo")
    ax.set_ylabel("Salário Médio")
    return pn.pane.Matplotlib(fig, tight=True, sizing_mode='stretch_width')

def grafico_distribuicao_salario():
    df = pd.read_sql_query("SELECT salario FROM pessoa", cnx)
    fig, ax = plt.subplots()
    ax.hist(df["salario"], bins=10, color='#17becf')
    ax.set_title("Distribuição de Salários")
    ax.set_xlabel("Salário")
    ax.set_ylabel("Quantidade")
    return pn.pane.Matplotlib(fig, tight=True, sizing_mode='stretch_width')


In [ ]:
def create_graficos_view():
    graficos_layout = pn.Column(
        pn.pane.Markdown("### 📊 Análise de Funcionários"),
        grafico_salario_por_sexo,
        grafico_distribuicao_salario,
        create_btn_voltar(),
        sizing_mode='stretch_width',
        margin=(20,20,20,20)
    )
    
    return graficos_layout

In [ ]:
def go_to_graficos():
    main_area.clear()
    main_area.append(create_graficos_view())

In [ ]:
# Conecta os botões da Home às funções de navegação
btn_go_crud.on_click(lambda event: go_to_crud())
btn_go_graficos.on_click(lambda event: go_to_graficos())

In [ ]:
# Inicializa com a tela Home
go_to_home()

In [ ]:
# App principal, com título e área dinâmica
pn.Column(
    pn.pane.Markdown("# 🏢 Sistema de Funcionários"),
    main_area
).servable()